In [3]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json
import numpy as np

from functions import (get_sector, changes_from_press, get_content_xom,
                       get_df, normalise_to_index, save_data)
import dateutil.parser

columns = ['date', 'press_title', 'urls', '1d_change', 'index_price','norm_price']
past_end_date = False

In [4]:
data = []
page_num = 1

ticker = 'xom'
sector = get_sector(ticker)

index_ticker = '^GSPC'

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)

# create break
while True:
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"}
    
    s = requests.Session()
    html = requests.get(f'https://investor.exxonmobil.com/news-events/press-releases?page={page_num}', headers=headers)


    soup = BeautifulSoup(html.content) 

    articles = soup.find_all('div',attrs={'class':'media-body col-md'})
    
    
    if page_num == 1:
        first_articles = articles
    else:
        if articles == first_articles:
            break    
    
    for article in articles:
        date = article.find('div', attrs={'class':'date'}).text.lstrip()
        title = article.find('a').text.lstrip().rstrip() 
        url = article.find('a').get('href')
        
        date = date.lstrip().rstrip().replace(',', '')    # remove starting and trailing whitespaces        
        date = dateutil.parser.parse(date).date()
        
        
        content = get_content_xom(url)
        
        if date < datetime(2014, 1, 1).date():
            past_end_date = True
            break
            
        pct_change = changes_from_press(stock_hist, date, 1)
        
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        data.append([str(date), title, content, url, pct_change, index_price, norm_price])
    if past_end_date: break

    page_num = page_num + 1
    
save_data(data, ticker, sector)

C:\Users\Aidan\AppData\Local\Programs\Python\Python310\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname CDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
C:\Users\Aidan\AppData\Local\Programs\Python\Python310\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname CST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [5]:
page_num

181